<a href="https://colab.research.google.com/github/WilderJoseth/Coursera_Capstone/blob/master/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Project
In this work I will use borough of Toronto.

###1. Data
This data comes from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M saving it in a CSV file/

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [12]:
ds = pd.read_csv('sample_data/torontoCodPostalList.csv')
ds.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
ds = ds[ds['Borough'] != 'Not assigned']
ds.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
ds.shape

(103, 3)